In [2]:
import os, shutil
import pandas as pd
import os
import re
import string
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import statistics
from scipy.stats.mstats import gmean

In [206]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic

In [4]:
import os
cwd = os.getcwd()

Cleaning up the data about the power plants 

In [5]:
gen=pd.read_csv(cwd+"//data//Generator.csv")
gen_prop=pd.read_csv(cwd+"//data//Generator_proposed.csv")
PP=pd.read_csv(cwd+"//data//PP_loc.csv")

/var/folders/gz/h80vdt3j7bv8pst2_nh54_k80000gn/T/ipykernel_58821/2308173538.py:1: DtypeWarning: Columns (0,9,13,14,15,17,18,34,35,36,37,38,39,40,41,42,47,48,49,50,51,52,53,62,63,67,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  gen=pd.read_csv(cwd+"//data//Generator.csv")


In [6]:
Gen_tot =  pd.concat([gen, gen_prop], ignore_index=True)
Gen_tot =Gen_tot.drop_duplicates(subset=['Plant Code', 'Generator ID', ], keep='last')

In [8]:
PP.columns
PP = PP[['Plant Code', 'Plant Name', 'Latitude',
       'Longitude']]
PP_merged = pd.merge(Gen_tot, PP, on = ['Plant Name','Plant Code'])

In [10]:
PP_of_interest = 'Nuclear|Natural Gas|Coal|Onshore Wind|Solar Photovoltaic|Hydro'

Plants = PP_merged['Technology'].str.contains(PP_of_interest, case=False, na=False)
PP_merged[Plants]=PP_merged[Plants][PP_merged[Plants]['Technology'].str.contains("Other|Storage| Gasification ") == False]


(19079, 80)

In [11]:
PP_merged[Plants].columns
PP_merged= PP_merged[PP_merged['Status'].str.contains("RE|SE|SB")==False]

In [12]:
df2 = PP_merged[Plants][[ 'Plant Code', 'Generator ID', 'Plant Name', 'Latitude', 'Longitude','Technology','Nameplate Capacity (MW)', 
       'Operating Month','Operating Year', 'Planned Retirement Month', 'Planned Retirement Year',
     'Current Month', 'Current Year']]
df2=df2[df2['Technology'].notna()]
df2.columns = ['Plant Code', 'Generator ID', 'name', 'Latitude', 'Longitude','source','capacity', 
       'Operating Month','Operating Year', 'Planned Retirement Month', 'Planned Retirement Year',
     'Current Month', 'Current Year']

/var/folders/gz/h80vdt3j7bv8pst2_nh54_k80000gn/T/ipykernel_58821/2568835521.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = PP_merged[Plants][[ 'Plant Code', 'Generator ID', 'Plant Name', 'Latitude', 'Longitude','Technology','Nameplate Capacity (MW)',


In [13]:
df2['Operating Year'].fillna(df2['Current Year'], inplace=True)
del df2['Current Year']
df2['Operating Month'].fillna(df2['Current Month'], inplace=True)
del df2['Current Month']

In [14]:
df2=df2.assign(coordinate= "("+df2['Latitude'].astype(str)+','+df2['Longitude'].astype(str)+")")
del df2['Latitude']
del df2['Longitude']

In [15]:
df2 = df2.replace(' ',np.nan)
df2['Planned Retirement Month'].fillna(df2['Operating Month'], inplace=True)

,Plant Code,Generator ID,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate
4,1.0,WT1,Sand Point,Onshore Wind Turbine,0.5,10.0,2011.0,10.0,NaN,"(55.339722,-160.497222)"
5,1.0,WT2,Sand Point,Onshore Wind Turbine,0.5,10.0,2011.0,10.0,NaN,"(55.339722,-160.497222)"
6,2.0,1,Bankhead Dam,Conventional Hydroelectric,53.9,7.0,1963.0,7.0,NaN,"(33.458665,-87.356823)"
7,3.0,1,Barry,Natural Gas Steam Turbine,153.1,2.0,1954.0,2.0,NaN,"(31.0069,-88.0103)"
8,3.0,2,Barry,Natural Gas Steam Turbine,153.1,7.0,1954.0,7.0,NaN,"(31.0069,-88.0103)"
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SRPAR,SR Paris,Solar Photovoltaic,6.8,10,2022,10,NaN,"(36.445894,-88.327688)"
26156,65642.0,SRJA2,SR Jackson II,Solar Photovoltaic,1,8,2022,8,NaN,"(35.607757,-88.917046)"
26157,65644.0,LUMII,Lumina II Solar Project,Solar Photovoltaic,357.8,6,2023,6,NaN,"(32.883528,-100.746389)"
26158,65645.0,LUMIN,Lumina Solar Project,Solar Photovoltaic,357.8,6,2023,6,NaN,"(32.901081,-100.982695)"


In [16]:
df2.loc[df2['source'].str.contains('Gas'), 'source'] = 'gas'
df2.loc[df2['source'].str.contains('Coal'), 'source'] = 'coal'
df2.loc[df2['source'].str.contains('Wind'), 'source'] = 'wind'
df2.loc[df2['source'].str.contains('Solar'), 'source'] = 'solar'
df2.loc[df2['source'].str.contains('Hydro'), 'source'] = 'hydro'
df2.loc[df2['source'].str.contains('Nuclear'), 'source'] = 'nuclear'

In [17]:
df2=df2.assign(name= df2['name'].astype(str)+'_'+df2['Generator ID'].astype(str))
del df2['Generator ID']

In [18]:
df2.name = df2.name.str.replace(' ','_')

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,NaN,"(55.339722,-160.497222)"
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,NaN,"(55.339722,-160.497222)"
6,2.0,Bankhead_Dam_1,hydro,53.9,7.0,1963.0,7.0,NaN,"(33.458665,-87.356823)"
7,3.0,Barry_1,gas,153.1,2.0,1954.0,2.0,NaN,"(31.0069,-88.0103)"
8,3.0,Barry_2,gas,153.1,7.0,1954.0,7.0,NaN,"(31.0069,-88.0103)"
...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022,10,NaN,"(36.445894,-88.327688)"
26156,65642.0,SR_Jackson_II_SRJA2,solar,1,8,2022,8,NaN,"(35.607757,-88.917046)"
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023,6,NaN,"(32.883528,-100.746389)"
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023,6,NaN,"(32.901081,-100.982695)"


In [19]:
def my_func(row):
    if row['source'] == 'nuclear':
        val = 60.0
    elif row['source']  == 'solar':
        val = 25.0
    elif row['source']  == 'wind':
        val = 20.0
    elif row['source']  == 'coal':
        val = 30.0        
    elif row['source']  == 'gas':
        val = 30.0          
    else:
        val = 40.0
    return val

df2['plant_life'] = df2.apply(my_func, axis=1)
df2

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,NaN,"(55.339722,-160.497222)",20.0
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,NaN,"(55.339722,-160.497222)",20.0
6,2.0,Bankhead_Dam_1,hydro,53.9,7.0,1963.0,7.0,NaN,"(33.458665,-87.356823)",40.0
7,3.0,Barry_1,gas,153.1,2.0,1954.0,2.0,NaN,"(31.0069,-88.0103)",30.0
8,3.0,Barry_2,gas,153.1,7.0,1954.0,7.0,NaN,"(31.0069,-88.0103)",30.0
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022,10,NaN,"(36.445894,-88.327688)",25.0
26156,65642.0,SR_Jackson_II_SRJA2,solar,1,8,2022,8,NaN,"(35.607757,-88.917046)",25.0
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023,6,NaN,"(32.883528,-100.746389)",25.0
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023,6,NaN,"(32.901081,-100.982695)",25.0


In [20]:
df2['Operating Year'] = df2['Operating Year'].astype(float)
df2['plant_life'] = df2['plant_life'].astype(float)
df2['Retirement'] = df2['Operating Year'] +df2['plant_life']

In [21]:
df2['Planned Retirement Year'].fillna(df2['Retirement'], inplace=True)
del df2['Retirement']
df2

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,2031.0,"(55.339722,-160.497222)",20.0
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,2031.0,"(55.339722,-160.497222)",20.0
6,2.0,Bankhead_Dam_1,hydro,53.9,7.0,1963.0,7.0,2003.0,"(33.458665,-87.356823)",40.0
7,3.0,Barry_1,gas,153.1,2.0,1954.0,2.0,1984.0,"(31.0069,-88.0103)",30.0
8,3.0,Barry_2,gas,153.1,7.0,1954.0,7.0,1984.0,"(31.0069,-88.0103)",30.0
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022.0,10,2047.0,"(36.445894,-88.327688)",25.0
26156,65642.0,SR_Jackson_II_SRJA2,solar,1,8,2022.0,8,2047.0,"(35.607757,-88.917046)",25.0
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023.0,6,2048.0,"(32.883528,-100.746389)",25.0
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023.0,6,2048.0,"(32.901081,-100.982695)",25.0


In [190]:
df2['Planned Retirement Year'] = df2['Planned Retirement Year'].astype(int)
plants = df2.loc[df2['Planned Retirement Year'] > 2019] 
plants['capacity'] = plants['capacity'].astype(str)
plants['capacity'] = plants['capacity'].apply(lambda x: x.replace(',', '')) 
plants['capacity'] = plants['capacity'].astype(float)

/var/folders/gz/h80vdt3j7bv8pst2_nh54_k80000gn/T/ipykernel_58821/3412394840.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plants['capacity'] = plants['capacity'].astype(str)
/var/folders/gz/h80vdt3j7bv8pst2_nh54_k80000gn/T/ipykernel_58821/3412394840.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plants['capacity'] = plants['capacity'].apply(lambda x: x.replace(',', ''))
/var/folders/gz/h80vdt3j7bv8pst2_nh54_k80000gn/T/ipykernel_58821/3412394840.py:5: SettingWithCopyWarning: 
A value is trying to 

Create a material flow for different categories of plants 

material streams data read

In [151]:
energy_data = pd.read_csv(cwd+"//data//model_data_f.csv")

power plants data

In [152]:
plants

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,2031,"(55.339722,-160.497222)",20.0
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,2031,"(55.339722,-160.497222)",20.0
11,3.0,Barry_A1CT,gas,170.1,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
12,3.0,Barry_A1CT2,gas,170.1,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
13,3.0,Barry_A1ST,gas,195.2,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022.0,10,2047,"(36.445894,-88.327688)",25.0
26156,65642.0,SR_Jackson_II_SRJA2,solar,1.0,8,2022.0,8,2047,"(35.607757,-88.917046)",25.0
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023.0,6,2048,"(32.883528,-100.746389)",25.0
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023.0,6,2048,"(32.901081,-100.982695)",25.0


# The Plan

We have to divide stations into ones that require materials for operation or not. 
Wind, solar, hydro do not require fuel constantly. They are one-shot build. Everything else requires constant refueling. 
Aggregation is annual. 

All stations build before 2019 (included) do not have any mining waste.
For those built after 2020 (included), we need to calculate mining waste. 
- How much **front_need, front_waste, back_waste** necessary?
    - Filter energy_data by stream, then group energy_data by source and aggregate over amount (kg/MW). For each plant, the front_need amount necessary is its capacity * aggregate with corresponding source.
- How much **oper_need, oper_waste** necessary?
    - Energy [MWh] = Capacity * 365 * 24 * Capacity factor [Solar = 0.25, Nuclear = 0.90, etc]. Then multiply on aggregate to amount (kg/MWh)
    - Time interval of interest in 2020- 2050. In 2020 we generates "this" much front need/ waste, oper need/waste, back need/waste.


### Generate aggregation per stream

In [154]:
streams = energy_data.loc[:, ~energy_data.columns.str.startswith('Unnamed:')].groupby(['stream', 'source'])

calculate the sum per stream (can we calculate each material amount too if needed?)

In [155]:
streams_agg = streams.sum()
streams_agg = pd.DataFrame(streams_agg.to_records())
streams_agg

,stream,source,amount (kg/MW),amount (kg/MWh)
0,back_waste,coal,2.103808e+05,1.143623
1,back_waste,gas,1.292360e+05,0.578548
2,back_waste,hydro,8.004730e+06,43.931827
3,back_waste,nuclear,2.042597e+05,0.431802
4,back_waste,solar,1.476455e+05,3.745446
5,back_waste,wind,1.746663e+04,0.398782
6,front_need,coal,2.109717e+05,1.146835
7,front_need,gas,1.293910e+05,0.579242
8,front_need,hydro,7.706588e+06,42.295555
9,front_need,nuclear,2.503413e+05,0.529218


Add amount of materials needed per MW capacity (a constant number that dependent of the source)

In [156]:
plants_capacity = plants[['name', 'source', 'capacity']]
merged = pd.merge(streams_agg, plants_capacity, on = ['source'])
merged

,stream,source,amount (kg/MW),amount (kg/MWh),name,capacity
0,back_waste,coal,210380.8467,1.143623,Whelan_Energy_Center_2,248.0
1,back_waste,coal,210380.8467,1.143623,Cross_1,590.9
2,back_waste,coal,210380.8467,1.143623,Cross_3,591.0
3,back_waste,coal,210380.8467,1.143623,Cross_4,652.0
4,back_waste,coal,210380.8467,1.143623,Seminole_(FL)_1,714.6
...,...,...,...,...,...,...
67420,oper_waste,wind,3304.0000,0.075434,"Bluestone_Wind,_LLC_BLUES",111.8
67421,oper_waste,wind,3304.0000,0.075434,Baron_Winds_II_BRNW2,113.2
67422,oper_waste,wind,3304.0000,0.075434,Number_Three_Wind_Project_65013,106.0
67423,oper_waste,wind,3304.0000,0.075434,Inertia_Wind_Project_WINER,301.0


If I need a separate data on materials 

In [157]:

merged_mat = pd.merge(streams, plants_capacity, on = ['source'])
merged_mat

TypeError: Can only merge Series or DataFrame objects, a <class 'pandas.core.groupby.generic.DataFrameGroupBy'> was passed

The amount per actual capacity 

In [158]:
merged["Energy (kg/MW)"] = merged["capacity"] * merged["amount (kg/MW)"]

Amount per KWh (wrong data, may delete)

In [159]:
merged["Energy (kg/MWh)"] = merged["capacity"] * merged["amount (kg/MWh)"]

Proly, need to rename the following variables as years

In [160]:
operating_plants = plants[['name', 'source', 'capacity', 'Operating Year']]
retiring_plants = plants[['name', 'source', 'capacity', 'Planned Retirement Year']]

summarise back waste over each year for all streams

In [161]:
back = pd.merge(merged.loc[merged['stream'].isin(['back_waste'])], retiring_plants, on=['name', 'source', 'capacity']).groupby(['Planned Retirement Year', 'source']).sum()
back.drop(columns=['amount (kg/MW)', 'amount (kg/MWh)', 'capacity'], inplace=True)
back[-20:]


Energy (kg/MW)  Energy (kg/MWh)
Planned Retirement Year source                                  
2055                    solar      3.289542e+07       834.485478
2056                    gas        5.148502e+08      2304.817877
                        hydro      3.053805e+09     16759.991906
                        nuclear    2.593894e+08       548.345518
2057                    hydro      1.658580e+09      9102.674503
2058                    gas        1.422500e+08       636.807329
                        hydro      1.082240e+09      5939.582977
2059                    hydro      1.496885e+08       821.525160
2060                    hydro      1.237531e+09      6791.860416
2061                    hydro      5.211079e+08      2859.961922
2062                    hydro      2.201301e+08      1208.125236
2063                    hydro      3.562105e+08      1954.966290
2064                    hydro      6.563879e+07       360.240979
2065                    hydro      6.932096e+08      3804.496197
2066                    hydro      1.397626e+09      7670.496951
2067                    hydro      8.084778e+07       443.711450
2076                    nuclear    2.593894e+08       548.345518
2083                    nuclear    4.550906e+08       962.055133
2089                    nuclear    1.572800e+07        33.248764
2090                    nuclear    7.863998e+07       166.243818

summarise front waste over each year for all streams

In [162]:
front = pd.merge(merged.loc[merged['stream'].isin(['front_need', 'front_waste'])], operating_plants, on=['name', 'source', 'capacity']).groupby(['Operating Year', 'stream', 'source']).sum()
front.drop(columns=['amount (kg/MW)', 'amount (kg/MWh)', 'capacity'], inplace=True)
front[:4]

Energy (kg/MW)  Energy (kg/MWh)
Operating Year stream      source                                 
1901.0         front_need  hydro     1.849581e+07       101.509331
               front_waste hydro     9.330149e+06        51.206031
1904.0         front_need  hydro     1.155988e+07        63.443332
               front_waste hydro     5.831343e+06        32.003770

Start calculating the operational streams that depend on the capacity factors and capacity

In [163]:
capacity_factor = {
    "nuclear" : 0.9,
    "wind" : 0.4,
    "coal" : 0.7,
    "solar" : 0.18,
    "hydro" : 0.52,
    "gas" : 0.85,
    }

Here we calculate operational streams in KG ("Materials") based on the plant capacity and capacity factors we have mentioned above

In [164]:
streams_agg_oper = streams_agg.loc[streams_agg['stream'].isin(['oper_need', 'oper_waste'])].drop(columns=['amount (kg/MW)'])
oper_merged = pd.merge(streams_agg_oper, plants_capacity, on = ['source'])
oper_merged["Materials"] = oper_merged["capacity"] * oper_merged['amount (kg/MWh)'] * 365 * 24 * oper_merged['source'].replace(capacity_factor)
oper_merged

,stream,source,amount (kg/MWh),name,capacity,Materials
0,oper_need,coal,545.000000,Whelan_Energy_Center_2,248.0,8.288011e+08
1,oper_need,coal,545.000000,Cross_1,590.9,1.974752e+09
2,oper_need,coal,545.000000,Cross_3,591.0,1.975087e+09
3,oper_need,coal,545.000000,Cross_4,652.0,2.178945e+09
4,oper_need,coal,545.000000,Seminole_(FL)_1,714.6,2.388150e+09
...,...,...,...,...,...,...
25435,oper_waste,wind,0.075434,"Bluestone_Wind,_LLC_BLUES",111.8,2.955098e+04
25436,oper_waste,wind,0.075434,Baron_Winds_II_BRNW2,113.2,2.992102e+04
25437,oper_waste,wind,0.075434,Number_Three_Wind_Project_65013,106.0,2.801792e+04
25438,oper_waste,wind,0.075434,Inertia_Wind_Project_WINER,301.0,7.956032e+04


In [165]:
oper = pd.merge(oper_merged, operating_plants, on=['name', 'source', 'capacity'])
oper["Interest Year"] = oper.apply(lambda row: 2020 if row["Operating Year"] <= 2020.0 else row["Operating Year"], axis=1)
oper = oper.drop(columns=['Operating Year'])
oper = oper.groupby(['Interest Year', 'stream', 'source']).sum()
oper = oper.drop(columns=['capacity', 'amount (kg/MWh)'])
oper 

Materials
Interest Year stream     source               
2020.0        oper_need  coal     3.395408e+11
                         gas      5.839084e+11
                         nuclear  7.677217e+07
                         solar    1.301957e+07
                         wind     3.102208e+07
              oper_waste coal     1.837950e+12
                         gas      1.507264e+12
                         nuclear  7.670970e+07
                         solar    1.301957e+07
                         wind     3.102208e+07
2021.0        oper_need  gas      9.367592e+09
                         solar    3.684070e+06
                         wind     3.885636e+06
              oper_waste gas      2.418091e+10
                         solar    3.684070e+06
                         wind     3.885636e+06
2022.0        oper_need  gas      1.029708e+10
                         solar    6.243704e+06
                         wind     2.747183e+06
              oper_waste gas      2.658023e+10
                         solar    6.243704e+06
                         wind     2.747183e+06
2023.0        oper_need  gas      1.079399e+10
                         nuclear  1.711163e+06
                         solar    5.982411e+06
                         wind     1.033861e+06
              oper_waste gas      2.786293e+10
                         nuclear  1.709771e+06
                         solar    5.982411e+06
                         wind     1.033861e+06
2024.0        oper_need  gas      2.503085e+09
                         solar    2.930044e+06
                         wind     9.676755e+05
              oper_waste gas      6.461305e+09
                         solar    2.930044e+06
                         wind     9.676755e+05
2025.0        oper_need  gas      7.697110e+09
                         solar    4.042301e+05
                         wind     8.569254e+05
              oper_waste gas      1.986883e+10
                         solar    4.042301e+05
                         wind     8.569254e+05
2026.0        oper_need  gas      4.890757e+09
                         solar    2.895641e+05
                         wind     4.285684e+05
              oper_waste gas      1.262469e+10
                         solar    2.895641e+05
                         wind     4.285684e+05
2028.0        oper_need  gas      8.749348e+08
                         solar    1.525920e+04
              oper_waste gas      2.258501e+09
                         solar    1.525920e+04
2029.0        oper_need  nuclear  5.913805e+04
              oper_waste nuclear  5.908993e+04
2030.0        oper_need  nuclear  2.956903e+05
                         solar    6.181363e+04
              oper_waste nuclear  2.954496e+05
                         solar    6.181363e+04

## Operation materials need and waste from 2020 to 2050

In [166]:
years_of_interest = [(x, y, z) for x in range(2020, 2051) for y in ["oper_need", "oper_waste"] for z in ["coal", "gas", "nuclear", "solar", "wind"]] 
years_of_interest_dict = {}
for item in years_of_interest:
    years_of_interest_dict[item] = 0.0

oper_over_years = pd.DataFrame({"Materials over Years" : years_of_interest_dict})
oper_over_years

Materials over Years
2020 oper_need  coal                      0.0
                gas                       0.0
                nuclear                   0.0
                solar                     0.0
                wind                      0.0
...                                       ...
2050 oper_waste coal                      0.0
                gas                       0.0
                nuclear                   0.0
                solar                     0.0
                wind                      0.0

[310 rows x 1 columns]

In [167]:
plants_lifetime = plants[['name', 'source', 'capacity', 'Operating Year', 'Planned Retirement Year']]
plants_lifetime

,name,source,capacity,Operating Year,Planned Retirement Year
4,Sand_Point_WT1,wind,0.5,2011.0,2031
5,Sand_Point_WT2,wind,0.5,2011.0,2031
11,Barry_A1CT,gas,170.1,2000.0,2030
12,Barry_A1CT2,gas,170.1,2000.0,2030
13,Barry_A1ST,gas,195.2,2000.0,2030
...,...,...,...,...,...
26155,SR_Paris_SRPAR,solar,6.8,2022.0,2047
26156,SR_Jackson_II_SRJA2,solar,1.0,2022.0,2047
26157,Lumina_II_Solar_Project_LUMII,solar,357.8,2023.0,2048
26158,Lumina_Solar_Project_LUMIN,solar,357.8,2023.0,2048


In [168]:
merged_over_years = pd.merge(oper_merged, plants_lifetime, on=['name', 'source', 'capacity'])
merged_over_years.rename(columns={"Materials":"mpy", "Operating Year":"oy", "Planned Retirement Year":"pry"}, inplace=True)
merged_over_years["oy"] = merged_over_years.apply(lambda row: 2020 if row["oy"] <= 2020.0 else row["oy"], axis=1)
merged_over_years["yrs"] = merged_over_years.apply(lambda r: [year for year in range(int(r['oy']), r['pry'] + 1)], axis=1)
merged_over_years.drop(columns=['amount (kg/MWh)', 'capacity', 'oy', 'pry'], inplace=True)
merged_over_years


,stream,source,name,mpy,yrs
0,oper_need,coal,Whelan_Energy_Center_2,8.288011e+08,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."
1,oper_waste,coal,Whelan_Energy_Center_2,4.486339e+09,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."
2,oper_need,coal,Cross_1,1.974752e+09,"[2020, 2021, 2022, 2023, 2024, 2025]"
3,oper_waste,coal,Cross_1,1.068943e+10,"[2020, 2021, 2022, 2023, 2024, 2025]"
4,oper_need,coal,Cross_3,1.975087e+09,"[2020, 2021, 2022, 2023, 2024, 2025, 2026, 202..."
...,...,...,...,...,...
25435,oper_waste,wind,Number_Three_Wind_Project_65013,2.801792e+04,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202..."
25436,oper_need,wind,Inertia_Wind_Project_WINER,7.956032e+04,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202..."
25437,oper_waste,wind,Inertia_Wind_Project_WINER,7.956032e+04,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202..."
25438,oper_need,wind,Clearwater_Wind_I_CW1,9.660896e+04,"[2022, 2023, 2024, 2025, 2026, 2027, 2028, 202..."


In [169]:
lst = [(stream, source, mpy, yrs) for stream, source, mpy, yrs in zip(merged_over_years['stream'], merged_over_years['source'], merged_over_years['mpy'], merged_over_years['yrs'])]

In [145]:
for item in lst:
    stream = item[0]
    source = item[1]
    materials_per_year = item[2]
    yrs = item[3]
    for yr in yrs:
        if yr > 2050: 
            continue
        oper_over_years["Materials over Years"][yr][stream][source] += materials_per_year

In [170]:
oper_over_years

Materials over Years
2020 oper_need  coal                      0.0
                gas                       0.0
                nuclear                   0.0
                solar                     0.0
                wind                      0.0
...                                       ...
2050 oper_waste coal                      0.0
                gas                       0.0
                nuclear                   0.0
                solar                     0.0
                wind                      0.0

[310 rows x 1 columns]

## Calculating emissions based on mileage of delivering front_need

In [178]:
plants

,Plant Code,name,source,capacity,Operating Month,Operating Year,Planned Retirement Month,Planned Retirement Year,coordinate,plant_life
4,1.0,Sand_Point_WT1,wind,0.5,10.0,2011.0,10.0,2031,"(55.339722,-160.497222)",20.0
5,1.0,Sand_Point_WT2,wind,0.5,10.0,2011.0,10.0,2031,"(55.339722,-160.497222)",20.0
11,3.0,Barry_A1CT,gas,170.1,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
12,3.0,Barry_A1CT2,gas,170.1,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
13,3.0,Barry_A1ST,gas,195.2,5.0,2000.0,5.0,2030,"(31.0069,-88.0103)",30.0
...,...,...,...,...,...,...,...,...,...,...
26155,65641.0,SR_Paris_SRPAR,solar,6.8,10,2022.0,10,2047,"(36.445894,-88.327688)",25.0
26156,65642.0,SR_Jackson_II_SRJA2,solar,1.0,8,2022.0,8,2047,"(35.607757,-88.917046)",25.0
26157,65644.0,Lumina_II_Solar_Project_LUMII,solar,357.8,6,2023.0,6,2048,"(32.883528,-100.746389)",25.0
26158,65645.0,Lumina_Solar_Project_LUMIN,solar,357.8,6,2023.0,6,2048,"(32.901081,-100.982695)",25.0


In [194]:
plant_coords = plants
plant_coords = plant_coords.drop(columns=['Plant Code', 'Operating Month', 'Planned Retirement Month', 'Planned Retirement Year', 'plant_life'])
plant_coords.rename(columns={"coordinate":"Plant Coordinate"}, inplace=True)
plant_coords

,name,source,capacity,Operating Year,Plant Coordinate
4,Sand_Point_WT1,wind,0.5,2011.0,"(55.339722,-160.497222)"
5,Sand_Point_WT2,wind,0.5,2011.0,"(55.339722,-160.497222)"
11,Barry_A1CT,gas,170.1,2000.0,"(31.0069,-88.0103)"
12,Barry_A1CT2,gas,170.1,2000.0,"(31.0069,-88.0103)"
13,Barry_A1ST,gas,195.2,2000.0,"(31.0069,-88.0103)"
...,...,...,...,...,...
26155,SR_Paris_SRPAR,solar,6.8,2022.0,"(36.445894,-88.327688)"
26156,SR_Jackson_II_SRJA2,solar,1.0,2022.0,"(35.607757,-88.917046)"
26157,Lumina_II_Solar_Project_LUMII,solar,357.8,2023.0,"(32.883528,-100.746389)"
26158,Lumina_Solar_Project_LUMIN,solar,357.8,2023.0,"(32.901081,-100.982695)"


In [195]:
material_coords = energy_data
material_coords = material_coords.loc[material_coords['stream'].isin(['front_need'])]
material_coords.drop(columns=['Unnamed: 6', 'amount (kg/MWh)', 'stream'], inplace=True)
material_coords.rename(columns={"coordinate":"Material Coordinate"}, inplace=True)
material_coords

/var/folders/gz/h80vdt3j7bv8pst2_nh54_k80000gn/T/ipykernel_58821/1826711988.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  material_coords.drop(columns=['Unnamed: 6', 'amount (kg/MWh)', 'stream'], inplace=True)
/var/folders/gz/h80vdt3j7bv8pst2_nh54_k80000gn/T/ipykernel_58821/1826711988.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  material_coords.rename(columns={"coordinate":"Material Coordinate"}, inplace=True)


,source,material,amount (kg/MW),Material Coordinate
3,gas,iron,3.143800e+04,"(31.0069,-88.0103)"
4,gas,concrete,9.774900e+04,"(31.0069,-88.0103)"
5,gas,aluminum,2.040000e+02,"(31.0069,-88.0103)"
38,hydro,aluminum,1.585210e+03,"(59.4545,-135.3131)"
39,hydro,concrete,7.644000e+06,"(55.615208,-131.356111)"
...,...,...,...,...
200,nuclear,cadmium,1.600000e-01,"(39.101667,-108.345556)"
250,nuclear,natural_uranium,1.500000e+04,"(39.18,-76.5389)"
251,nuclear,zirconium,8.041680e+02,"(28.4698,-80.7644)"
252,nuclear,iron,3.027456e+04,"(28.4698,-80.7644)"


In [214]:
coord_merged = pd.merge(plant_coords, material_coords, on=['source'])
coord_merged["Plant Coordinate"] = coord_merged["Plant Coordinate"].apply(lambda tup_s: tuple(float(x) for x in tup_s[1:-1].split(',')))
coord_merged["Material Coordinate"] = coord_merged["Material Coordinate"].apply(lambda tup_s: tuple(float(x) for x in tup_s[1:-1].split(',')))
coord_merged["Distance"] = coord_merged.apply(lambda row: geodesic(row["Plant Coordinate"], row["Material Coordinate"]).miles, axis=1)
coord_merged[:29]

,name,source,capacity,Operating Year,Plant Coordinate,material,amount (kg/MW),Material Coordinate,Distance
0,Sand_Point_WT1,wind,0.5,2011.0,"(55.339722, -160.497222)",aluminum,8.026800e+03,"(36.190278, -95.2894)",3275.608462
1,Sand_Point_WT1,wind,0.5,2011.0,"(55.339722, -160.497222)",copper,5.237760e+01,"(36.190278, -95.2894)",3275.608462
2,Sand_Point_WT1,wind,0.5,2011.0,"(55.339722, -160.497222)",zinc,2.698240e+01,"(34.4341, -92.9049)",3456.301998
3,Sand_Point_WT1,wind,0.5,2011.0,"(55.339722, -160.497222)",iron,4.735040e+04,"(40.45193, -121.86215)",2031.689901
4,Sand_Point_WT1,wind,0.5,2011.0,"(55.339722, -160.497222)",concrete,2.246400e+06,"(35.462382, -93.804926)",3369.783350
5,Sand_Point_WT1,wind,0.5,2011.0,"(55.339722, -160.497222)",copper,5.568000e+03,"(40.1719, -88.8339)",3334.873234
6,Sand_Point_WT1,wind,0.5,2011.0,"(55.339722, -160.497222)",fiberglass,3.490800e+03,"(38.23926, -95.68978)",3160.646484
7,Sand_Point_WT1,wind,0.5,2011.0,"(55.339722, -160.497222)",iron,5.407100e+05,"(40.570817, -122.02093)",2019.977931
8,Sand_Point_WT1,wind,0.5,2011.0,"(55.339722, -160.497222)",paint,1.311120e+03,"(40.570817, -122.02093)",2019.977931
9,Sand_Point_WT1,wind,0.5,2011.0,"(55.339722, -160.497222)",polyethylene,3.294000e+02,"(40.7415, -124.2103)",1928.540270


In [220]:
coord_merged = coord_merged.drop(columns=['Plant Coordinate', 'Material Coordinate'])
coord_merged

KeyError: "['Plant Coordinate', 'Material Coordinate'] not found in axis"

In [218]:
def emission_factor():
    return 2

def truck_factor():
    return 1/4000

In [223]:
coord_merged["Emission"] = emission_factor() * coord_merged["capacity"] * coord_merged["amount (kg/MW)"] * coord_merged["Distance"] * truck_factor()
coord_merged = coord_merged.loc[coord_merged['Operating Year'] > 2019]
emission = coord_merged.groupby(['Operating Year']).sum("Emission")
emission

,capacity,amount (kg/MW),Distance,Emission
Operating Year,,,,
2020.0,431210.2,4.668303e+08,1.408269e+07,1.816040e+10
2021.0,462178.7,4.344571e+08,1.212799e+07,1.771854e+10
2022.0,479538.4,3.360129e+08,1.177975e+07,1.431694e+10
2023.0,389781.2,1.770176e+08,3.873011e+06,8.088733e+09
2024.0,195248.0,1.389415e+08,1.408368e+06,5.264745e+09
2025.0,99160.0,1.202197e+08,4.778398e+05,5.356631e+09
2026.0,56010.4,3.903792e+08,1.503408e+06,4.125751e+09
2027.0,40.4,1.541318e+07,1.281397e+04,5.987995e+06
2028.0,2645.0,3.426765e+05,1.014148e+04,5.800327e+07
